In [ ]:
from fenics import *
from numpy.random import random
set_log_level(30)
import matplotlib.pyplot as plt
from mshr import *

In [ ]:
class TuringPattern(NonlinearProblem):
    def __init__(self, a, L):
        NonlinearProblem.__init__(self)
        self.L = L
        self.a = a
    def F(self, b, x): assemble(self.L, tensor=b)
    def J(self, A, x): assemble(self.a, tensor=A)

In [ ]:
#mesh = UnitSquareMesh(48, 48)

p0 = Point(0.0, 0.0)
p1 = Point(3.0,3.0) 

channel = Rectangle(p0, p1)
C_1 = Circle(Point(0.5, 0.5),  0.3, 15)
C_2 = Circle(Point(3, 3),  1.0, 15)

domain = channel - C_1 - C_2 

mesh = generate_mesh(domain,64)
plot(mesh)

In [ ]:
U = FiniteElement("CG", mesh.ufl_cell(), 2)
W = FunctionSpace(mesh, U * U)

dtrial   = TrialFunction(W)
q, p = TestFunctions(W)

w = Function(W)
w0 =  Function(W)

# Split mixed functions
du, dv = split(dtrial)
u, v = split(w)
u0, v0 = split(w0)

dt = 2
T = 100

In [ ]:
class IC(UserExpression):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def eval(self, values, x):
        values[0] = 1.0*random() +0.25
        values[1] = 1.0*random() +0.25
    def value_shape(self): return (2,)

In [ ]:
w_init = IC(element=W.ufl_element(), degree=2)
w.interpolate(w_init)
w0.interpolate(w_init)

In [ ]:
p1 = plot(u0)
p1.set_cmap("seismic")
plt.title("$u(t=0)$")
plt.colorbar(p1) 
plt.savefig("Turing_0.png")
plt.show()  

p2 = plot(v0)
p2.set_cmap("gray")
plt.title("$v(t=0)$")
plt.colorbar(p2) 
#plt.savefig("Turing_v0.png")
plt.show()

In [ ]:
D_u = 0.001
D_v = 0.005

k = -0.05 
tau = 0.1

L0 = u*q - u0*q  + D_u*inner(grad(u), grad(q))*dt - u*q*dt + u*u*u*q*dt + v*q*dt - k*q*dt
L1 = tau*v*p - tau*v0*p + D_v*inner(grad(v), grad(p))*dt - u*p*dt +v*p*dt 
L  = (L0 + L1) *dx

In [ ]:
# Compute directional derivative about u in the direction of du
a = derivative(L, w, dtrial)

problem = TuringPattern(a, L)
solver = NewtonSolver()
solver.parameters["linear_solver"] = "lu"
solver.parameters["convergence_criterion"] = "incremental"
solver.parameters["relative_tolerance"] = 1e-2

In [ ]:
t = 0
while t < T:
    t += dt
    w0.vector()[:] = w.vector()
    solver.solve(problem, w.vector())

    u,v = w.split()
    
    p_u = plot(u)
    p_u.set_cmap("seismic")
    plt.title("u(t = {})".format(round(t,2)))
    plt.colorbar(p_u)   
    plt.savefig("Turing_{}.png".format(t)) 
    plt.show()  

    p_v = plot(v)
    p_v.set_cmap("gray")
    plt.title("v(t = {})".format(round(t,2)))
    plt.colorbar(p_v)   
    #plt.savefig("Turing_{}.png".format(t)) 
    plt.show()      